[![Roboflow Notebooks](https://media.roboflow.com/notebooks/template/bannertest2-2.png?ik-sdk-version=javascript-1.4.3&updatedAt=1672932710194)](https://github.com/roboflow/notebooks)

# Working with PPM Image Format in Autodistill

---

## Steps in this Tutorial

In this tutorial, we are going to cover:

- Understanding PPM (Portable Pixmap Format) images
- Creating PPM images for testing
- Loading PPM images with Autodistill's `load_image` function
- Converting PPM images to other formats
- Using PPM images in an Autodistill workflow

## Introduction to PPM Format

PPM (Portable Pixmap Format) is a simple image format that stores RGB pixels in an array. It's part of the Netpbm family of image formats, which also includes PBM (Portable Bitmap) and PGM (Portable Graymap).

PPM is a lossless format that doesn't use compression, which makes it ideal for testing and development. Files in PPM format can be either:
- ASCII format (P3) - Human readable but inefficient
- Binary format (P6) - More efficient storage

While PPM files are larger than formats like JPEG or PNG due to lack of compression, they're useful in certain scientific and academic contexts where data integrity is crucial.

## Installation

First, let's make sure we have the required libraries installed:

In [ ]:
!pip install autodistill supervision numpy opencv-python pillow -q

## Import Required Libraries

In [ ]:
import os
import numpy as np
import cv2
from PIL import Image
import supervision as sv
from autodistill.helpers import load_image

## Creating a PPM Image

Let's start by creating a simple PPM image for testing purposes. We'll create both an ASCII (P3) format and a binary (P6) format PPM file.

In [ ]:
def create_ascii_ppm(filepath="test_ascii.ppm", width=100, height=100):
    """Create a simple ASCII PPM image file (P3 format)"""
    with open(filepath, 'w') as f:
        f.write('P3\n')  # ASCII format
        f.write(f'{width} {height}\n')
        f.write('255\n')  # Max color value
        
        # Write RGB data as ASCII numbers
        for y in range(height):
            for x in range(width):
                # Create a gradient
                r = (x * 255) // width
                g = (y * 255) // height
                b = ((x+y) * 255) // (width+height)
                f.write(f"{r} {g} {b} ")
            f.write('\n')
    
    return filepath

def create_binary_ppm(filepath="test_binary.ppm", width=100, height=100):
    """Create a simple binary PPM image file (P6 format)"""
    with open(filepath, 'wb') as f:
        f.write(b'P6\n')
        f.write(f'{width} {height}\n'.encode())
        f.write(b'255\n')  # Max color value
        
        # Write RGB data as binary
        for y in range(height):
            for x in range(width):
                # Create a gradient
                r = (x * 255) // width
                g = (y * 255) // height
                b = ((x+y) * 255) // (width+height)
                f.write(bytes([r, g, b]))
    
    return filepath

# Create both types of PPM files
ascii_ppm = create_ascii_ppm()
binary_ppm = create_binary_ppm()

print(f"Created ASCII PPM: {ascii_ppm}")
print(f"Created Binary PPM: {binary_ppm}")

## Loading PPM Images with Autodistill

Autodistill's `load_image` function now supports PPM files directly. Let's load both of our test files:

In [ ]:
# Load the ASCII PPM
ascii_image_pil = load_image(ascii_ppm, return_format="PIL")
ascii_image_cv2 = load_image(ascii_ppm, return_format="cv2")
ascii_image_numpy = load_image(ascii_ppm, return_format="numpy")

# Load the Binary PPM
binary_image_pil = load_image(binary_ppm, return_format="PIL")
binary_image_cv2 = load_image(binary_ppm, return_format="cv2")
binary_image_numpy = load_image(binary_ppm, return_format="numpy")

# Display information about the loaded images
print("ASCII PPM loaded as:")
print(f"  PIL Image: {type(ascii_image_pil)} with size {ascii_image_pil.size} and mode {ascii_image_pil.mode}")
print(f"  CV2 Image: {type(ascii_image_cv2)} with shape {ascii_image_cv2.shape}")
print(f"  Numpy Array: {type(ascii_image_numpy)} with shape {ascii_image_numpy.shape}")
print("\nBinary PPM loaded as:")
print(f"  PIL Image: {type(binary_image_pil)} with size {binary_image_pil.size} and mode {binary_image_pil.mode}")
print(f"  CV2 Image: {type(binary_image_cv2)} with shape {binary_image_cv2.shape}")
print(f"  Numpy Array: {type(binary_image_numpy)} with shape {binary_image_numpy.shape}")

## Visualizing the PPM Images

Let's visualize our PPM images using both Matplotlib and Supervision's `plot_image` function:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.title('ASCII PPM (P3 Format)')
plt.imshow(ascii_image_pil)
plt.axis('off')

plt.subplot(1, 2, 2)
plt.title('Binary PPM (P6 Format)')
plt.imshow(binary_image_pil)
plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
print("Using Supervision to display the PPM images:")
print("\nASCII PPM Image:")
sv.plot_image(ascii_image_cv2, size=(6, 6))

print("\nBinary PPM Image:")
sv.plot_image(binary_image_cv2, size=(6, 6))

## Converting Between PPM and Other Formats

Autodistill and PIL make it easy to convert between PPM and other commonly used formats:

In [ ]:
# Convert PPM to JPEG
binary_image_pil.save("converted_from_ppm.jpg")

# Convert PPM to PNG
binary_image_pil.save("converted_from_ppm.png")

# Convert JPEG to PPM
test_jpg = Image.new('RGB', (100, 100), color='red')
test_jpg.save("test.jpg")
jpg_image = Image.open("test.jpg")
jpg_image.save("converted_to_ppm.ppm")

print("Conversion complete. Files created:")
print("- converted_from_ppm.jpg")
print("- converted_from_ppm.png")
print("- test.jpg")
print("- converted_to_ppm.ppm")

## Using PPM Images in Autodistill Workflow

Now let's demonstrate how to use PPM images directly in an Autodistill workflow. First, let's create a more complex test image that has enough features to detect:

In [ ]:
# Let's create a PPM with some simple shapes
def create_shapes_ppm(filepath="shapes.ppm", width=640, height=480):
    # Create a blank canvas
    img = np.zeros((height, width, 3), dtype=np.uint8)
    
    # Draw a red rectangle
    img[50:150, 100:200] = [0, 0, 255]  # Red in BGR
    
    # Draw a green circle
    cv2.circle(img, (width//2, height//2), 100, (0, 255, 0), -1)  # Green circle
    
    # Draw a blue triangle
    pts = np.array([[450, 50], [550, 150], [400, 200]], np.int32)
    pts = pts.reshape((-1, 1, 2))
    cv2.fillPoly(img, [pts], (255, 0, 0))  # Blue triangle
    
    # Save as PPM
    cv2_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    cv2_pil.save(filepath)
    
    return filepath

# Create and display the shapes image
shapes_ppm = create_shapes_ppm()
shapes_img = load_image(shapes_ppm, return_format="cv2")
print(f"Created shapes PPM image: {shapes_ppm}")
sv.plot_image(shapes_img, size=(10, 8))

Now let's set up a simple Autodistill workflow using a mock model to demonstrate loading and processing PPM images:

In [ ]:
# Simple mock model to detect colored regions
# (In a real scenario, you would use a proper model like YOLO or GroundingDINO)
def detect_colored_regions(image_path):
    # This is a mock implementation to simulate model detection
    img = load_image(image_path, return_format="cv2")
    
    # Convert to HSV for easier color detection
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Define color thresholds
    lower_red = np.array([0, 100, 100])
    upper_red = np.array([10, 255, 255])
    lower_green = np.array([50, 100, 100])
    upper_green = np.array([70, 255, 255])
    lower_blue = np.array([110, 100, 100])
    upper_blue = np.array([130, 255, 255])
    
    # Create masks and find contours
    detections = []
    
    # Red objects
    red_mask = cv2.inRange(hsv, lower_red, upper_red)
    red_contours, _ = cv2.findContours(red_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in red_contours:
        if cv2.contourArea(cnt) > 1000:  # Filter small contours
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append(([x, y, x+w, y+h], 0.9, 0))  # class_id 0 for red
    
    # Green objects
    green_mask = cv2.inRange(hsv, lower_green, upper_green)
    green_contours, _ = cv2.findContours(green_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in green_contours:
        if cv2.contourArea(cnt) > 1000:  # Filter small contours
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append(([x, y, x+w, y+h], 0.95, 1))  # class_id 1 for green
    
    # Blue objects
    blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
    blue_contours, _ = cv2.findContours(blue_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for cnt in blue_contours:
        if cv2.contourArea(cnt) > 1000:  # Filter small contours
            x, y, w, h = cv2.boundingRect(cnt)
            detections.append(([x, y, x+w, y+h], 0.85, 2))  # class_id 2 for blue
    
    return detections

# Process our shapes PPM image
detections = detect_colored_regions(shapes_ppm)

# Convert to Supervision Detections format
xyxy = np.array([det[0] for det in detections])
confidence = np.array([det[1] for det in detections])
class_id = np.array([det[2] for det in detections])

sv_detections = sv.Detections(
    xyxy=xyxy,
    confidence=confidence,
    class_id=class_id
)

# Define class names
class_names = ["red", "green", "blue"]

# Visualize the detections
box_annotator = sv.BoxAnnotator()

# Create labels for each detection
labels = [f"{class_names[class_id]} {confidence:.2f}" for class_id, confidence in zip(sv_detections.class_id, sv_detections.confidence)]

# Annotate the image
annotated_frame = box_annotator.annotate(
    scene=shapes_img.copy(), 
    detections=sv_detections,
    labels=labels
)

# Display the result
sv.plot_image(annotated_frame, size=(10, 8))

## Processing Multiple Images Including PPM Files

Now let's demonstrate how to process a mixed directory containing PPM, JPG, and PNG files:

In [ ]:
# Create a directory to store mixed format images
import shutil

if not os.path.exists('mixed_images'):
    os.makedirs('mixed_images')

# Copy existing files
shutil.copy(shapes_ppm, 'mixed_images/shapes.ppm')
shutil.copy('converted_from_ppm.jpg', 'mixed_images/gradient.jpg')
shutil.copy('converted_from_ppm.png', 'mixed_images/gradient.png')

# Process all images in the directory
def process_image_directory(dir_path):
    all_files = os.listdir(dir_path)
    image_files = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg', '.png', '.ppm', '.pbm', '.pgm'))]
    
    print(f"Found {len(image_files)} images in {dir_path}: {image_files}")
    
    for image_file in image_files:
        image_path = os.path.join(dir_path, image_file)
        print(f"\nProcessing {image_file}...")
        
        # Load the image using Autodistill's load_image function
        try:
            img = load_image(image_path, return_format="cv2")
            print(f"  Successfully loaded with shape {img.shape}")
            
            # For demonstration purposes, just compute average color
            avg_color = np.mean(img, axis=(0, 1))
            print(f"  Average BGR color: {avg_color}")
            
            # In a real application, you would run your model here
            # detections = model.predict(img)
            
        except Exception as e:
            print(f"  Error processing {image_file}: {e}")

# Process our mixed directory
process_image_directory('mixed_images')

## Converting Between Image Formats in Autodistill

Finally, let's implement a simple function to convert images between formats while preserving their contents:

In [ ]:
def convert_image(input_path, output_path):
    """Convert an image from one format to another.
    
    Args:
        input_path: Path to the input image
        output_path: Path to the output image (extension determines format)
    """
    # Use Autodistill's load_image to open the image (supports PPM)
    img_pil = load_image(input_path, return_format="PIL")
    
    # Save in the new format
    img_pil.save(output_path)
    
    print(f"Converted {input_path} to {output_path}")
    return output_path

# Convert our shapes.ppm to PNG
convert_image(shapes_ppm, "shapes_converted.png")

# Convert the binary PPM to JPEG
convert_image(binary_ppm, "binary_converted.jpg")

# Load and display one of the converted images to confirm it works
converted_img = load_image("shapes_converted.png", return_format="cv2")
sv.plot_image(converted_img, size=(8, 6))

## Working with PPM Format in `split_data` Function

The Autodistill library's `split_data` function now supports PPM files. Let's demonstrate how it handles PPM images during dataset preparation:

In [ ]:
from autodistill.helpers import split_data

# Create a mini dataset with PPM images
if not os.path.exists('mini_dataset'):
    os.makedirs('mini_dataset')
    os.makedirs('mini_dataset/images')
    os.makedirs('mini_dataset/annotations')

# Create a few test PPM images
create_shapes_ppm('mini_dataset/images/shape1.ppm', 320, 240)
create_shapes_ppm('mini_dataset/images/shape2.ppm', 320, 240)
create_shapes_ppm('mini_dataset/images/shape3.ppm', 320, 240)

# Create mock annotation files
for i in range(1, 4):
    with open(f'mini_dataset/annotations/shape{i}.txt', 'w') as f:
        f.write(f"0 0.5 0.5 0.2 0.2\n")  # Class 0, center x, center y, width, height

# Create a data.yaml file
with open('mini_dataset/data.yaml', 'w') as f:
    f.write("names:\n  - shape\n")

# Now run split_data
split_data('mini_dataset', split_ratio=0.67)

Let's verify what happened with our PPM files after running `split_data`:

In [ ]:
# Check the directory structure
print("Directory structure after split_data:")
!ls -R mini_dataset

print("\nNote that PPM files were converted to JPG format during the split_data process, which is expected behavior.")
print("This conversion ensures compatibility with models that may not support PPM directly.")

## Cleanup

Let's clean up the files we created:

In [ ]:
# Remove created test files
import shutil

files_to_remove = [
    "test_ascii.ppm", 
    "test_binary.ppm", 
    "converted_from_ppm.jpg", 
    "converted_from_ppm.png",
    "test.jpg", 
    "converted_to_ppm.ppm", 
    "shapes.ppm", 
    "shapes_converted.png", 
    "binary_converted.jpg"
]

for file in files_to_remove:
    if os.path.exists(file):
        os.remove(file)
        print(f"Removed {file}")

# Remove directories
dirs_to_remove = ["mixed_images", "mini_dataset"]

for dir_path in dirs_to_remove:
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print(f"Removed directory {dir_path}")

## Conclusion

In this tutorial, we've demonstrated how to work with PPM image format in Autodistill:

1. We created PPM images in both ASCII (P3) and binary (P6) formats
2. We loaded PPM images using Autodistill's `load_image` function with various return formats (PIL, cv2, numpy)
3. We visualized PPM images using both Matplotlib and Supervision's `plot_image`
4. We converted PPM images to other formats (JPEG, PNG) and vice versa
5. We used PPM images in a simple object detection workflow
6. We processed a directory containing mixed image formats including PPM files
7. We demonstrated how `split_data` handles PPM files by converting them to JPG

The PPM format is now fully supported throughout the Autodistill library, making it possible to work with this format alongside more common formats like JPEG and PNG.